### Given a topic research the latest news

In [ ]:
import sys

sys.path.append("..")

from dotenv import load_dotenv

load_dotenv()



In [ ]:
from utils import utilsk

In [ ]:
utils.SCRIPT = "outputs/robertson/script.yaml"
scenes = utils.get_scenes()
# Extract lines starting with "[Scene"
scene_lines = [s.description for s in scenes]
scene_lines

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained(
    "SG161222/Realistic_Vision_V2.0", torch_dtype=torch.float16
)

In [ ]:
pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
pipeline.scheduler.compatibles

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
positive_prompt = """
(high detailed skin:1.2, high quality, reportage, journalism),
8k uhd, dslr, soft lighting, film grain, Fujifilm XT3
"""

negative_prompt = """
((text:1.5, deformed iris, deformed pupils, deformed hands, nsfw)),
((semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4)),
close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate,
morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation,
deformed, blurry, burns, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured,
gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs,
fused fingers, too many fingers, long neck
"""

In [ ]:
guidance_scale = 0.7
num_inference_steps = 50
num_images_per_prompt = 5
image_height = 512
image_width = 768

for i, line in enumerate(scene_lines):
    prompt = line
    images = pipeline(
        prompt="{} {}".format(prompt, positive_prompt),
        negative_prompt=negative_prompt,
        height=image_height,
        width=image_width,
        num_images_per_prompt=num_images_per_prompt,
        num_inference_steps=num_inference_steps,
        # guidance_scale=guidance_scale,
    ).images

    for j, image in enumerate(images):
        image.save("outputs/robertson/scene_{}_{}.png".format(i + 1, j + 1))

In [ ]:
# Free GPU memory for upscaler
del pipeline
del images
torch.cuda.empty_cache()

In [ ]:
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline

import torch

pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    revision="fp16",
    torch_dtype=torch.float16,
)

In [ ]:
from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention(
    attention_op=MemoryEfficientAttentionFlashAttentionOp
)
# Workaround for not accepting attention shape using VAE for Flash Attention
pipeline.vae.enable_xformers_memory_efficient_attention(attention_op=None)

In [ ]:
pipeline.scheduler.compatibles

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
guidance_scale = 0.7
num_inference_steps = 20
num_images_per_prompt = 1
n = 5

for i, prompt in enumerate(scene_lines):
    for j in range(1, n + 1):
        low_res_image = Image.open(
            "outputs/robertson/scene_{}_{}.png".format(i + 1, j)
        ).convert("RGB")
        low_res_image = low_res_image.resize((256, 256))

        image = pipeline(
            prompt="{} {}".format(prompt, positive_prompt),
            negative_prompt=negative_prompt,
            image=low_res_image,
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            # guidance_scale=guidance_scale,
        ).images[0]

        image.save("outputs/robertson/scene_{}_{}_upscaled.png".format(i + 1, j))

## LORAs

In [1]:
import sys

sys.path.append("..")

from dotenv import load_dotenv

load_dotenv()
from IPython.display import display, Image

In [2]:
from utils import image_gen
import os

In [3]:
sd_model = "SG161222/Realistic_Vision_V3.0"
#sd_model = "Lykon/dreamshaper"

In [4]:
import torch
from safetensors.torch import load_file
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

In [5]:
pipeline = StableDiffusionPipeline.from_pretrained(sd_model, torch_dtype=torch.float16, safety_checker=None)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

In [9]:
pipeline.load_lora_weights("../loras", weight_name="ACTOR_Heather.safetensors")

In [10]:
pipeline.to('cuda')

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.19.3",
  "_name_or_path": "SG161222/Realistic_Vision_V3.0",
  "feature_extractor": [
    null,
    null
  ],
  "requires_safety_checker": null,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "DPMSolverMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [11]:
pipeline.enable_attention_slicing()
pipeline.enable_vae_slicing()

In [12]:
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
prompt = "ACTOR_Heather in a forest, wearing green pantsuit, high quality"
negative_prompt = ("(low quality, worst quality:1.4), (bad anatomy), (inaccurate limb:1.2), low resolution"
                   "bad composition, inaccurate eyes, extra digit, deformed face, fewer digits, (extra arms:1.2), large breasts")

images = pipeline(prompt=prompt, 
    negative_prompt=negative_prompt, 
    width=512, 
    height=512, 
    num_inference_steps=30, 
    num_images_per_prompt=4,
    generator=torch.manual_seed(0)
).images

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
display(*images)